In [1]:
import os, torch
os.chdir("/Users/ESKO/PYWORK/NHITS")
print(os.getcwd())

C:\Users\ESKO\anaconda3\envs\dl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\ESKO\PYWORK\NHITS


In [2]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from ray import tune
from sklearn.preprocessing import StandardScaler

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoTFT
from neuralforecast.models import NBEATS, NBEATSx, NHITS, TFT
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, GMM, PMM
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.losses.pytorch import MAE

## <기준날짜 및 예측 기간 설정>

In [5]:
cutoff_date = '2021-12-30' ## len = 65
test_period = 12 ## 예측 기간 (월)
corr_rate = 0.8 ## Lag corr rate

## <원 data set>

In [6]:
df = pd.read_csv('./data/metal_pst.csv')
df.rename(columns={'date':'ds'},inplace=True)
df.rename(columns={'li2co3_cif_fast_kg_spot_exchng':'y'},inplace=True)
df['ds'] = pd.to_datetime(df['ds'], format='%Y-%m-%d')

raw_y_set_resampled = df.copy()
raw_y_set_resampled.rename(columns={'ds':'date'},inplace=True)
raw_y_set_resampled.rename(columns={'lioh_cif_fast_kg_spot_exchng':'y2'},inplace=True)
del raw_y_set_resampled['unique_id']

## Feature shift 12개월

In [26]:
%%time

raw_y_set_resampled = raw_y_set_resampled[raw_y_set_resampled.date<=cutoff_date]
raw_y_set_shifted_merged = raw_y_set_resampled['date']

for i in range(len(raw_y_set_resampled.columns)):
    if i == len(raw_y_set_resampled.columns) - 1:
        break
    raw_y_set_shifted = raw_y_set_resampled.iloc[:,[0,i+1]].dropna()
    for j in range(10,13):
        raw_y_set_shifted = pd.concat([raw_y_set_shifted, (raw_y_set_shifted.iloc[:,1].shift(j+1).rename(raw_y_set_resampled.columns[i+1] + '_' +str(j+1)))],axis=1)
        raw_y_set_shifted_merged = pd.merge(raw_y_set_shifted_merged, raw_y_set_shifted, how= 'outer') 
        if j == len(raw_y_set_shifted.iloc[:,1]) -2 :
            break
# raw_y_set_shifted_merged.to_csv('./data/raw_y_set_shifted_merged.csv', encoding='cp949')

CPU times: total: 2min 13s
Wall time: 4min 13s


#### 해당 기간에서의 lag 12 best

In [27]:
raw_y_set_shifted_merged_corr = raw_y_set_shifted_merged.corr()
raw_y_set_shifted_merged_filtered = raw_y_set_shifted_merged_corr[raw_y_set_resampled.columns.drop('date')]
raw_y_set_shifted_merged_filtered.sort_values('y',ascending=False).head(50)
# raw_y_set_shifted_merged_filtered = raw_y_set_shifted_merged_filtered.loc[raw_y_set_shifted_merged_filtered.index != 'y']
# raw_y_set_shifted_merged_filtered = raw_y_set_shifted_merged_filtered.loc[raw_y_set_shifted_merged_filtered.index != 'y2']
# raw_y_set_shifted_merged_filtered_y = raw_y_set_shifted_merged_filtered.iloc[:,:1]
# raw_y_set_shifted_merged_filtered_90 = raw_y_set_shifted_merged_filtered_y[raw_y_set_shifted_merged_filtered_y.y >= corr_rate]
# raw_y_set_shifted_merged_filtered_90 = raw_y_set_shifted_merged_filtered_90[raw_y_set_shifted_merged_filtered_90.index.str.contains('_12')]
# futr_list = raw_y_set_shifted_merged_filtered_90.index.values.tolist()
# futr_list = [word.strip('_12') for word in futr_list ]

# futr_list.remove('y')
# futr_list.remove('y2')

# futr_list

,y,y2,Baltic Dry Index (BDI),Howe Robinson Container Index,Shanghai Containerized Freight Index (SCFI),옥수수 For Grain [미국(생산자 가격)] 현물,원당 (daily) [ISA] 현물,대두 [미국(생산자 가격)] 현물,대두박 [파라나구아(FOB)] 현물,대두유 crude and refined average price [파라나구아(FOB)] 현물,...,Pilbara Mineral Limited_price_Open,Pilbara Mineral Limited_price_High,Pilbara Mineral Limited_price_Low,Pilbara Mineral Limited_price_Close,Pilbara Mineral Limited_price_Adj Close,Allkem Limited (AKE.AX)_price_Open,Allkem Limited (AKE.AX)_price_High,Allkem Limited (AKE.AX)_price_Low,Allkem Limited (AKE.AX)_price_Close,Allkem Limited (AKE.AX)_price_Adj Close
y,1.000000,0.968891,-0.036190,0.088180,0.099529,-0.055953,0.436810,0.716835,-0.137007,0.435993,...,0.314031,0.310957,0.313578,0.310818,0.395085,0.388181,0.383222,0.386912,0.384309,0.384309
lioh_exw_fast_kg,0.972708,0.969071,0.008285,0.112111,0.048538,-0.058438,0.520185,0.688471,-0.096422,0.448129,...,0.350605,0.347230,0.350220,0.347234,0.410575,0.441785,0.436631,0.439949,0.437363,0.437363
lioh_mag_am_kg,0.971355,0.969441,-0.031583,0.057918,0.010648,-0.109006,0.484097,0.697741,-0.111827,0.444239,...,0.315785,0.312194,0.315563,0.312327,0.373943,0.413135,0.407967,0.411488,0.408772,0.408772
y2,0.968891,1.000000,-0.132618,-0.018903,-0.051191,-0.157866,0.515224,0.685059,-0.214867,0.343290,...,0.199116,0.196053,0.198523,0.196018,0.270146,0.268934,0.263493,0.266758,0.264255,0.264255
li2co3_995_am_kg,0.936789,0.888275,0.048659,0.150128,0.139669,-0.003390,0.462890,0.692929,-0.020602,0.482094,...,0.385354,0.383537,0.384198,0.382773,0.459679,0.500542,0.496085,0.500175,0.497884,0.497884
li2co3_exw_fast_kg,0.935844,0.886636,0.076728,0.179863,0.159567,0.016121,0.477728,0.688053,-0.021214,0.491126,...,0.404742,0.402933,0.403541,0.402095,0.478054,0.514434,0.509927,0.513983,0.511742,0.511742
CLI_KOR_11,0.907573,0.883199,0.057960,0.031126,0.053052,-0.169190,0.118186,0.734802,-0.088989,0.644742,...,0.363763,0.357127,0.366902,0.359753,0.363727,0.486814,0.482371,0.488278,0.483710,0.483710
CPI_KOR_TOT_FOODENRG_AGRWTH,0.885160,0.901649,0.040553,0.136590,0.014163,0.065820,0.669488,0.536352,-0.042033,0.296458,...,0.313534,0.311713,0.312247,0.311196,0.382680,0.370328,0.366164,0.367100,0.366516,0.366516
CLI_KOR_12,0.866829,0.860532,-0.034392,-0.067899,-0.002984,-0.257107,0.009464,0.733218,-0.145449,0.612231,...,0.280886,0.273780,0.284298,0.276532,0.278487,0.395109,0.390934,0.396444,0.391807,0.391807
가성소다(caustic soda) 32% [중국] 현물,0.854719,0.908453,-0.234602,-0.197970,-0.223170,-0.368645,0.436017,0.641587,-0.363599,0.310409,...,-0.022302,-0.025920,-0.022874,-0.025781,0.036527,0.073023,0.067066,0.070311,0.068120,0.068120


In [21]:
raw_y_set_shifted_merged_filtered

,y,y2,Baltic Dry Index (BDI),Howe Robinson Container Index,Shanghai Containerized Freight Index (SCFI),옥수수 For Grain [미국(생산자 가격)] 현물,원당 (daily) [ISA] 현물,대두 [미국(생산자 가격)] 현물,대두박 [파라나구아(FOB)] 현물,대두유 crude and refined average price [파라나구아(FOB)] 현물,...,Pilbara Mineral Limited_price_Open,Pilbara Mineral Limited_price_High,Pilbara Mineral Limited_price_Low,Pilbara Mineral Limited_price_Close,Pilbara Mineral Limited_price_Adj Close,Allkem Limited (AKE.AX)_price_Open,Allkem Limited (AKE.AX)_price_High,Allkem Limited (AKE.AX)_price_Low,Allkem Limited (AKE.AX)_price_Close,Allkem Limited (AKE.AX)_price_Adj Close
y,1.000000,0.968891,-0.036190,0.088180,0.099529,-0.055953,0.436810,0.716835,-0.137007,0.435993,...,0.314031,0.310957,0.313578,0.310818,0.395085,0.388181,0.383222,0.386912,0.384309,0.384309
y_12,0.395666,0.506703,-0.606273,-0.660955,-0.672810,-0.659170,-0.222881,0.497372,-0.627526,0.042640,...,-0.474319,-0.480567,-0.472164,-0.478576,-0.475910,-0.397635,-0.402927,-0.398014,-0.402270,-0.402270
y2,0.968891,1.000000,-0.132618,-0.018903,-0.051191,-0.157866,0.515224,0.685059,-0.214867,0.343290,...,0.199116,0.196053,0.198523,0.196018,0.270146,0.268934,0.263493,0.266758,0.264255,0.264255
y2_12,0.459060,0.551905,-0.570005,-0.649912,-0.637150,-0.688537,-0.237037,0.509917,-0.629803,0.104790,...,-0.448634,-0.454282,-0.446880,-0.452525,-0.447282,-0.337842,-0.342994,-0.337249,-0.341535,-0.341535
Baltic Dry Index (BDI),-0.036190,-0.132618,1.000000,0.885149,0.636293,0.769650,0.235701,-0.202932,0.424479,0.356243,...,0.784158,0.784079,0.784514,0.784307,0.728007,0.709411,0.708686,0.710042,0.710111,0.710111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Allkem Limited (AKE.AX)_price_Low_12,0.045402,0.113927,-0.361162,-0.344744,-0.166817,-0.338400,-0.442696,0.510547,-0.516223,0.203860,...,-0.207755,-0.211547,-0.205810,-0.209931,-0.214979,-0.277925,-0.278463,-0.277998,-0.279505,-0.279505
Allkem Limited (AKE.AX)_price_Close,0.384309,0.264255,0.710111,0.779587,0.615391,0.639444,0.425993,0.185027,0.558216,0.580441,...,0.946203,0.945401,0.946003,0.945300,0.922718,0.999879,0.999934,0.999907,1.000000,1.000000
Allkem Limited (AKE.AX)_price_Close_12,0.047476,0.115951,-0.361204,-0.344576,-0.165832,-0.337482,-0.440134,0.511506,-0.514796,0.205382,...,-0.206401,-0.210178,-0.204446,-0.208556,-0.213339,-0.276506,-0.277020,-0.276553,-0.278039,-0.278039
Allkem Limited (AKE.AX)_price_Adj Close,0.384309,0.264255,0.710111,0.779587,0.615391,0.639444,0.425993,0.185027,0.558216,0.580441,...,0.946203,0.945401,0.946003,0.945300,0.922718,0.999879,0.999934,0.999907,1.000000,1.000000


## scasler

In [ ]:
# df_tgt = df.iloc[:,3:]

# scaler = StandardScaler()
# scaler.fit(df_tgt)
# arr = scaler.transform(df_tgt)
# df_tmp = pd.DataFrame(arr, columns=df_tgt.columns)
# df = pd.concat([df.iloc[:,:3],df_tmp], axis=1)

#### static DF

In [ ]:
static_df = pd.read_csv('/users/ESKO/PYWORK/NHITS/data/static_df.csv')

## <Training & Test>

#### 날짜 확인

In [146]:
# train = df.loc[df.ds < '2021-06-30']
# test = df.loc[df.ds > '2021-06-30']

#### 날짜 별 len 확인

In [ ]:
# len(df.loc[df.ds<'2021-06-30'])

In [ ]:
Y_train_df = df[df.ds<df['ds'].values[len(df.loc[df.ds<cutoff_date])]] # 132 train
Y_test_df = df[df.ds>=df['ds'].values[len(df.loc[df.ds<=cutoff_date])]][:test_period]

In [ ]:
futr_temp = Y_test_df.drop(['y'], axis=1).copy()
# futr_list = [
#              'Shanghai Containerized Freight Index (SCFI)',
#              '대두 [미국(생산자 가격)] 현물',
#              # '대두유 crude and refined average price [파라나구아(FOB)] 현물',
#              '보일러용 후판 Q245R 30mm Xinyu Steel [중국] 현물',
#              # 'HOUSECOST_KOR'
#              ]
futr_date = futr_temp.iloc[:,:2].copy()
futr_df = pd.concat([futr_date,futr_temp[futr_list]], axis=1)
# futr_df

In [ ]:
horizon = len(Y_test_df)

models = [
          # NBEATS(input_size=5 * horizon, gpus = 1, h=horizon, max_epochs=100),
          # NBEATSx(h=12, input_size=24,
          #       # loss=DistributionLoss(distribution='Normal', level=[80, 90]),
          #       gpus=1,
          #       scaler_type='robust',
          #       stat_exog_list = None,  
          #       futr_exog_list = futr_list,
          #       max_steps=100,
          #       # val_check_steps=10,
          #       # early_stop_patience_steps=2
          #      ),
          NHITS(input_size=5 * horizon, 
                h=horizon,
                gpus = 1,
                stat_exog_list = None,
                hist_exog_list=None,
                futr_exog_list = futr_list,
                # n_freq_downsample=[24, 12, 1],
                scaler_type = 'robust',
                learning_rate=1e-4,
                pooling_mode = 'MaxPool1d',
                activation='ReLU',
                batch_size=128,
                random_seed=42,
                max_epochs=200
               ),
           # NHITS(
           #       h,
           #       input_size,
           #       futr_exog_list=None,
           #       hist_exog_list=None,
           #       stat_exog_list=None,
           #       stack_types: list = ['identity', 'identity', 'identity'],
           #       n_blocks: list = [1, 1, 1],
           #       mlp_units: list = [[512, 512], [512, 512], [512, 512]],
           #       n_pool_kernel_size: list = [2, 2, 1],
           #       n_freq_downsample: list = [4, 2, 1],
           #       pooling_mode: str = 'MaxPool1d',
           #       interpolation_mode: str = 'linear',
           #       dropout_prob_theta=0.0,
           #       activation='ReLU',
           #       loss=MAE(),
           #       learning_rate=0.001,
           #       batch_size=32,
           #       windows_batch_size: int = 1024,
           #       step_size: int = 1,
           #       scaler_type='identity',
           #       random_seed=1,
           #       num_workers_loader=0,
           #       drop_last_loader=False,
           #       **trainer_kwargs,
    
          # TFT(h=12, 
          #     input_size=5 * horizon,
          #     # hidden_size=20,
          #     #loss=DistributionLoss(distribution='Poisson', level=[80, 90]),
          #     #loss=DistributionLoss(distribution='Normal', level=[80, 90]),
          #     #loss=DistributionLoss(distribution='StudentT', level=[80, 90]),
          #     learning_rate=0.005,
          #     stat_exog_list = None,
          #     futr_exog_list = futr_list,
          #     max_steps=100,
          #     gpus=1,
          #     # val_check_steps=10,
          #     # early_stop_patience_steps=10,
          #     scaler_type='robust',
          #     # windows_batch_size=None,
          #     enable_progress_bar=True),
             ]
nforecast = NeuralForecast(models=models, freq='M')
nforecast.fit(df=Y_train_df)
Y_hat_df = nforecast.predict(futr_df=futr_df).reset_index()

In [ ]:
# Plot predictions
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')

plot_df[['y', 'NHITS']].plot(ax=ax, linewidth=2, marker='o')
plt.axvline(cutoff_date, color='red')

ax.set_title('LI2CO3_PRICE', fontsize=22)
ax.set_ylabel('LI2CO3 Actual and Predicted Plot', fontsize=20)
ax.set_xlabel('Date', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

In [ ]:
plot_df[['y', 'NHITS']]